In [1]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install python-dotenv

In [6]:
import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [7]:
load_dotenv()
SPOTIFY_CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
SPOTIFY_SECRET_ID =  os.getenv('SPOTIFY_SECRET_ID')

In [8]:
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIFY_CLIENT_ID, client_secret=SPOTIFY_SECRET_ID)

In [11]:
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [12]:
playlist_link = 'https://open.spotify.com/playlist/37i9dQZEVXbMDoHDwVN2tF'

In [18]:
playlist_URI = playlist_link.split('/')[-1]

In [20]:
data = sp.playlist_tracks(playlist_URI)

In [25]:
data['items'][0]['track']['album']['id']

'10FLjwfpbxLmW8c25Xyc2N'

In [26]:
data['items'][0]['track']['album']['name']

'Die With A Smile'

In [27]:
data['items'][0]['track']['album']['release_date']

'2024-08-16'

In [28]:
data['items'][0]['track']['album']['total_tracks']

1

In [29]:
data['items'][0]['track']['album']['external_urls']['spotify']

'https://open.spotify.com/album/10FLjwfpbxLmW8c25Xyc2N'

In [33]:
album_list= []

In [34]:
for row in data['items']:
    album_id = row['track']['album']['id']
    album_name = row['track']['album']['name']
    album_release_date = row['track']['album']['release_date']
    album_total_tracks = row['track']['album']['total_tracks']
    album_url = row['track']['album']['external_urls']['spotify']
    album_element = {
        'album_id': album_id,
        'album_name': album_name, 
        'album_release_date': album_release_date,
        'album_total_tracks': album_total_tracks,
        'album_url': album_url
    }
    album_list.append(album_element)
